In [1]:
import pandas as pd
import numpy as np

In [9]:
rawdata = pd.read_csv('~/Desktop/uc2_rawdata_v2.csv', sep=',')
rawdata = rawdata.drop("Unnamed: 0", axis = 1)
rawdata.head()

,originalqueueid,UserId,caseType,Date_of_the_week,CaseCompletedPerMonth,CasesReopenedPerMonth,CasesReroutedPerMonth,UserCapacity,ExperienceInMonths,ResolutionTIme.WorkingTime,...,TYPE_RequestLicenseFile,TYPE_Return,TYPE_RequestPreviousVersionSerial.,TYPE_HULArequestorextension,TYPE_GeneralRegistrationHelp,TYPE_General,TYPE_StandaloneLicensingConfiguration,TYPE_DeploymentTroubleshooting,TYPE_Configuration.productinstalled.,TYPE_DownloadHelp
0,00G3A000004bYHTUA2,00530000005ff68AAA,Product Questions,3,0.569699,22.446135,0.0,2,8.776567,0.03,...,0,0,0,0,0,0,0,0,0,0
1,00G3A000004bYHSUA2,00530000005ff68AAA,Product Questions,4,0.113940,3.873952,0.0,2,8.776567,0.00,...,0,0,0,0,0,0,0,0,0,0
2,00G3A000004bYHSUA2,00530000003vGM0AAM,Product Questions,5,0.168426,2.021108,0.0,2,5.937338,0.13,...,0,0,0,0,0,0,0,0,0,0
3,00G3A000004bYHTUA2,00530000003vGM0AAM,Product Questions,5,0.336851,4.547492,0.0,2,5.937338,0.27,...,0,0,0,0,0,0,0,0,0,0
4,00G3A000004bYHTUA2,00530000005ff68AAA,Product Questions,6,0.569699,22.446135,0.0,2,8.776567,0.00,...,0,0,0,0,0,0,0,0,0,0


In [13]:
##categorical variables converts to Dummys
def make_dummies(df, variables):
    for variable in variables:
        dummies = pd.get_dummies(df[variable], prefix = variable)
        df = pd.concat([df, dummies], axis = 1)
        df = df.drop(variable, axis = 1)
    return df

final_data = make_dummies(rawdata, ["caseType", "originalqueueid", "UserId", "Date_of_the_week"])

In [14]:
final_data.shape

(1402, 159)

In [15]:
final_data.head(5)

,CaseCompletedPerMonth,CasesReopenedPerMonth,CasesReroutedPerMonth,UserCapacity,ExperienceInMonths,ResolutionTIme.WorkingTime,CaseResolutionTIme,UPS_X00e30000001UueC_263_21_04_15_257,UPS_Omni_Channel_Callback_Permission_Set,UPS_DocuSign_User,...,UserId_0053A00000Cp9NbQAJ,UserId_0053A00000Cp9PhQAJ,UserId_0053A00000CpCAzQAN,UserId_0053A00000CwCc4QAF,Date_of_the_week_2,Date_of_the_week_3,Date_of_the_week_4,Date_of_the_week_5,Date_of_the_week_6,Date_of_the_week_7
0,0.569699,22.446135,0.0,2,8.776567,0.03,1.773889,1,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.113940,3.873952,0.0,2,8.776567,0.00,0.004864,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.168426,2.021108,0.0,2,5.937338,0.13,0.134132,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.336851,4.547492,0.0,2,5.937338,0.27,0.963102,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.569699,22.446135,0.0,2,8.776567,0.00,0.004864,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [17]:
X_final = final_data.drop("ResolutionTIme.WorkingTime", axis=1)
y_target = pd.Series(final_data["ResolutionTIme.WorkingTime"])

In [34]:
y_target.shape

(1402,)

In [27]:
y_target_binary = pd.Series(np.where(final_data["ResolutionTIme.WorkingTime"] > 0.32, 1, 0))

In [33]:
y_target_binary.shape

(1402,)

In [36]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_target_binary, test_size=0.2, random_state=100)

In [39]:
y_train.shape

(1121,)

In [50]:
from sklearn import linear_model
logit_1 = linear_model.LogisticRegression(C=1e4)

In [51]:
logit_1.fit(X_train, y_train)
print ([logit_1.coef_, logit_1.intercept_])
print (logit_1.score(X_train, y_train))

[array([[ -1.59944836e-02,  -1.41987413e-02,   6.54447655e-02,
         -3.44189070e+00,  -2.46204264e-01,   1.69946723e+00,
         -1.82736630e-01,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,  -1.82736630e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,  -1.82736630e-01,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
       

In [61]:
logit_1.predict_proba(X_test)[:,1]

array([  5.64036438e-01,   2.52157516e-01,   2.82017345e-01,
         5.16075837e-01,   9.99757657e-01,   1.50870958e-01,
         1.79000679e-01,   1.33595674e-02,   8.24856036e-02,
         6.79145613e-02,   9.75588121e-01,   2.43047750e-01,
         2.44377196e-01,   3.88806386e-01,   1.02721264e-01,
         2.82485632e-01,   4.01975047e-01,   9.36157461e-01,
         5.34718332e-01,   4.18822202e-02,   1.83387890e-01,
         6.04172263e-01,   9.99313815e-01,   7.74692780e-01,
         8.86531117e-01,   9.99609211e-01,   9.99943342e-01,
         9.99999992e-01,   9.14447645e-01,   2.95740981e-07,
         2.04235667e-02,   9.99975188e-01,   4.86309231e-01,
         3.29141603e-01,   9.96958636e-01,   9.89347824e-01,
         9.97702579e-01,   2.06540357e-01,   9.99994472e-01,
         6.14179390e-01,   7.34133009e-01,   3.04441974e-01,
         9.63918494e-01,   4.41713580e-01,   2.98919596e-01,
         5.43398338e-01,   6.87481526e-02,   8.92792564e-02,
         1.03988945e-01,

In [63]:
from sklearn.metrics import roc_auc_score
print (roc_auc_score(y_train, logit_1.predict_proba(X_train)[:,1]))
print (roc_auc_score(y_test, logit_1.predict_proba(X_test)[:,1]))
print (roc_auc_score(y_target_binary, logit_1.predict_proba(X_final)[:,1]))

0.945364375434
0.883230034546
0.933098039695


In [68]:
pd.DataFrame(logit_1.predict_proba(X_final)[:,1])

,0
0,5.678295e-01
1,4.330340e-02
2,7.668068e-02
3,4.062376e-01
4,1.333425e-02
5,6.131920e-03
6,9.991370e-01
7,1.000000e+00
8,9.494419e-03
9,1.685594e-03
